In [32]:
generate_dataset(128, 64, 32000, 0, 'Inputs', rotate=False, y_translation=False, multi_fonts=False)

 99% (31999 of 32000) |################# | Elapsed Time: 0:52:24 ETA:   0:00:00

Saving File.........


In [33]:
generate_dataset(128, 64, 32000, 1, 'Inputs', rotate=False, y_translation=False, multi_fonts=False)

 99% (31999 of 32000) |################# | Elapsed Time: 0:53:59 ETA:   0:00:00

Saving File.........


In [34]:
generate_dataset(128, 64, 32000, 2, 'Inputs', rotate=False, y_translation=False, multi_fonts=False)

 99% (31998 of 32000) |################ | Elapsed Time: 21:50:25 ETA:   0:00:00

Saving File.........


In [95]:
img = paint_it('Hello', 128, 64, 'Inputs', script='E')
img = img[0]
img = np.transpose(img)
img = np.flip(img,0)
print(img.shape)
import numpy as np
import cv2
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

(128, 64)


In [29]:
def generate_dataset(w, h, samples, file_number, input_path, rotate=False, y_translation=False, multi_fonts=False):

    import pickle
    import numpy as np
    import progressbar
    
    x = []
    y = []
    
    f = open(input_path+'/words.pickle', 'rb')
    Eng_Corp =  pickle.load(f)
    f = open(input_path+'/words_hindi.pickle', 'rb')
    Hin_Corp =  pickle.load(f)
    
    word_types = [0,1] #mono-grams and bi-gram
    prb = [0.8,0.2]
    
    if w==64:
        length = [1, 3]
    elif w==128:
        length = [3, 6]
        prb = prb+[-0.3,0.3]
    elif w==256:
        length = [4, 12]
        prb = prb+[-0.2,0.2]
    elif w==512:
        length = [6, 22]
        prb = prb+[-0.2,0.2]
    else:
        length = [8, 40]
        prb = prb+[-0.2,0.2]
                    
    
    
    word_type_array = np.random.choice(word_types, samples, prb)
    
    bar = progressbar.ProgressBar(max_value=samples)
    
    for i in range(samples):

        language = np.random.randint(0,2)
        if language==0:
            text=''
            if word_type_array[i]==0:
                seed_point = np.random.randint(len(Eng_Corp['mono-grams']))
                text=Eng_Corp['mono-grams'][seed_point]
                trig=0
                while trig==0:
                    if len(text)>=length[0] and len(text)<=length[1]:
                        trig=1
                    else:
                        seed_point+=1
                        if seed_point>=len(Eng_Corp['mono-grams']):
                            seed_point=0
                        text = Eng_Corp['mono-grams'][seed_point]
                    
            else:
                seed_point = np.random.randint(len(Eng_Corp['bi-grams']))
                text=Eng_Corp['bi-grams'][seed_point]
                trig=0
                while trig==0:
                    if len(text)>=length[0] and len(text)<=length[1]:
                        trig=1
                    else:
                        seed_point+=1
                        if seed_point>=len(Eng_Corp['bi-grams']):
                            seed_point=0
                        text = Eng_Corp['bi-grams'][seed_point]
            #print(text)    
            x.append(paint_it(text, w, h, input_path, rotate=rotate, y_traslation=y_translation, multi_fonts=multi_fonts, script='E'))
            y.append(text)
                
        if language==1:
            text=''
            if word_type_array[i]==0:
                seed_point = np.random.randint(len(Hin_Corp['mono-grams']))
                text=Hin_Corp['mono-grams'][seed_point]
                trig=0
                while trig==0:
                    if len(text)>=length[0] and len(text)<=length[1]:
                        trig=1
                    else:
                        seed_point+=1
                        if seed_point>=len(Hin_Corp['mono-grams']):
                            seed_point=0
                        text = Hin_Corp['mono-grams'][seed_point]
                    
            else:
                seed_point = np.random.randint(len(Hin_Corp['bi-grams']))
                text=Hin_Corp['bi-grams'][seed_point]
                trig=0
                while trig==0:
                    if len(text)>=length[0] and len(text)<=length[1]:
                        trig=1
                    else:
                        seed_point+=1
                        if seed_point>=len(Hin_Corp['bi-grams']):
                            seed_point=0
                        text = Hin_Corp['bi-grams'][seed_point]
            img  = paint_it(text, w, h, input_path, rotate=rotate, y_traslation=y_translation, multi_fonts=multi_fonts, script='H')
            x.append(img)
            y.append(text)
            
        bar.update(i)
    print('Saving File.........')
    dictionary = {'X':np.array(x), 'Y':np.array(y)}
    
    if rotate==True:
        rotate_sign='Y'
    else:
        rotate_sign='N'
        
    if y_translation==True:
        y_translation_sign='Y'
    else:
        y_translation_sign='N'
        
    if multi_fonts==True:
        multi_fonts_sign='Y'
    else:
        multi_fonts_sign='N'
    
    pickle.dump( dictionary, open( "Inputs/HE set/"+str(w)+"-"+str(samples)+"-"+rotate_sign+"R-"+y_translation_sign+"Y-"+multi_fonts_sign+"F-"+str(file_number)+".pickle", "wb" ) )
    
    

In [23]:
paint_it('text', 128, 64, 'Inputs', rotate=False, y_traslation=False, multi_fonts=False, script='E').shape

(128, 64, 1)

In [25]:
def paint_it(text, w, h, input_path, rotate=False, y_traslation=False, multi_fonts=False, script='H'):

    from PIL import Image, ImageDraw, ImageFont
    from numpy import asarray
    import numpy as np
    from scipy import ndimage
    import os
    import math
    
    ratio = w/h
    #-------------------Canvas-----------------
    R = np.random.randint(0,255)
    G = np.random.randint(0,255)
    B = np.random.randint(0,255)
    base_color = int(0.21*R+0.72*G+0.07*B)
    surface = Image.new("RGBA", (w, h), color=(R,G,B,255))

    trig=0
    while trig==0:
    
        #-------------------Font Selection--------------------------------------

        if script=='E':
            path = input_path+'/fonts'
            final_text=text
        elif script=='H':
            path = input_path+'/Hindi_fonts/UTF'
            length_text = len(text)
            final_text = ''
            i=0
            while i<length_text-1:
                if text[i+1]==u'\u093F':
                    final_text+=u'\u093F'
                    final_text+=text[i]
                    i+=2
                else:
                    final_text+=text[i]
                    i+=1
            if len(final_text)<len(text):
                final_text+=text[-1]
        elif script=='G':
            path = input_path+'/Gujarati_fonts'
            finak_text=text

        if multi_fonts:
            font_selected = np.random.choice(os.listdir(path))
        else:
            font_selected = np.random.choice(os.listdir(path)[0:3])
            
        #-------------------Rotation Angle--------------------------------------------
        rotation_angle=0
        if rotate:
            rotation_angle = np.random.randint(50)-25
        if rotation_angle<=0:
            rad = (rotation_angle+360)*3.14/180
        else:
            rad = rotation_angle*3.14/180


        #-----------------------Font Size---------------------------------------------------

        font_size = 20
        f = ImageFont.truetype(path+'/'+font_selected, font_size)
        text_width, text_height = f.getsize(text)
        text_ratio = text_width/text_height
        #print(ratio, text_ratio, text_width, text_height)
        
        font_size=1

        if text_ratio>=ratio:         #text width is more
            subjected_width = text_width*abs(math.cos(rad))+text_height*abs(math.sin(rad))
            while subjected_width <= w-5:
                font_size+=1
                f = ImageFont.truetype(path+'/'+font_selected, font_size)
                text_width, text_height = f.getsize(text)
                subjected_width = text_width*abs(math.cos(rad))+text_height*abs(math.sin(rad))
            font_size-=2

        else:                         # text height is more
            subjected_height = text_height*abs(math.cos(rad))+text_width*abs(math.sin(rad))
            while subjected_height <= h-5:
                font_size+=1
                f = ImageFont.truetype(path+'/'+font_selected, font_size)
                text_width, text_height = f.getsize(text)
                subjected_height = text_height*abs(math.cos(rad))+text_width*abs(math.sin(rad))
            font_size-=2

        # ----------------------------- height and width after rotation ---------------------------------
        f = ImageFont.truetype(path+'/'+font_selected, font_size)
        text_width, text_height = f.getsize(text)
        final_height = text_height*abs(math.cos(rad))+text_width*abs(math.sin(rad))
        final_width = text_width*abs(math.cos(rad))+text_height*abs(math.sin(rad))
        
        #print(text, ' -- ', rotation_angle, ' -- ', font_selected, ' -- ',final_width,' -- ', final_height)
        if final_width<w-1 and final_height<h-1:
            trig=1
    #---------------------------DEbugger------------------------------------
    
    #print(text, ' -- ', rotation_angle, ' -- ', font_selected, ' -- ',final_width,' -- ', final_height)
    
    #------------------------------X-Y Trasnpose------------------------------------------------
    max_shift_x = w-final_width
    max_shift_y = h-final_height
    
    
    top_left_x = np.random.randint(0, int(max_shift_x))
    
    if y_traslation:
        top_left_y = np.random.randint(0, int(max_shift_y))
    else:
        top_left_y = 2    
        
    
    #--------------------------------Upper Canvas--------------------------------------------

    text_canvas = Image.new("RGBA", (int(final_width), int(final_height)), color=(R,G,B,255))
    draw = ImageDraw.Draw(text_canvas)
    
    difference = 0
    while difference==0:
        R_ = np.random.randint(0,255)
        G_ = np.random.randint(0,255)
        B_ = np.random.randint(0,255)
        font_color = int(0.21*R_+0.72*G_+0.07*B_)
        if abs(font_color-base_color)>=127:
            difference=1
    
    draw.text((0, 0), final_text, font=f, fill=(R_,G_,B_))
    
    #-------------------------------Text Rotation----------------------------------------------

    if rotate:                                                                   
        text_canvas = text_canvas.rotate(rotation_angle, expand=1)
  
    # -----------------------------Paste Canvas---------------------------------------------------

    sx, sy = text_canvas.size
    surface.paste(text_canvas, (top_left_x, top_left_y, top_left_x+sx, top_left_y+sy), text_canvas)
    
    #-----------------------------Binary of Image-----------------------------------------------

    surface = asarray(surface.convert('L'))
    
    surface = surface.astype(np.float32)/255

    #------------------------------Gaussian Noise------------------------------------------------

    severity = np.random.uniform(0, 0.6)
    blur = ndimage.gaussian_filter(np.random.randn(*surface.shape) * severity, 1)                        
    surface = (surface + blur)
    surface[surface > 1] = 1
    surface[surface <= 0] = 0
    
    #-------------------------------Inverse of image------------------------------------------------

    k = np.random.choice([0,1])
    if k==1:
        surface = 1-surface
    
    surface = np.transpose(surface)
    surface = np.flip(surface,0).reshape((w,h,1))
    
    return np.array(surface)
    

In [25]:
def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
    elif noise_typ == "s&p":
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
        out[coords] = 1

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
        out[coords] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy

In [ ]:
img = x[30]
img = img.reshape((64,128))
import numpy as np
import cv2
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()